# Model implementation

In [ ]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report, confusion_matrix,
    mean_squared_error, r2_score,
    silhouette_score, davies_bouldin_score
)
from sklearn.svm import SVC
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans

from mlxtend.frequent_patterns import apriori, association_rules


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Loading Dataset and preprocessing

In [ ]:

df = pd.read_csv("combined.csv")

df["year"] = df["date"].str.slice(0, 4).astype(int)
df = df[df["year"] >= 2010].copy()
df["fire_occurred"] = (df["FIRE_NAME"] != "no_fire").astype(int)

numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

print("Original data snapshot (after filtering 2010+):")
print(df.head())

Original data snapshot (after filtering 2010+):
         date                county   maxtempF   mintempF   avgtempF  \
1232  2010-01         Sierra County  36.612903  24.774194  31.774194   
1233  2010-01     Sacramento County  55.967742  43.935484  50.774194   
1234  2010-01  Santa Barbara County  63.483871  45.935484  55.870968   
1235  2010-01      Calaveras County  55.967742  34.258065  46.032258   
1236  2010-01        Ventura County  57.741935  42.483871  51.451613   

      totalSnow      humid      wind    precip  q_avgtempF  q_avghumid  \
1232   1.852934  92.709677  7.290323  0.329032   70.598566   60.243369   
1233   0.000000  76.516129  6.612903  0.348387   74.082437   55.555197   
1234   0.000000  62.483871  7.322581  0.296774   75.326882   54.421864   
1235   0.000000  72.161290  7.612903  0.200000   74.369892   50.507885   
1236   0.000000  64.903226  7.129032  0.312903   67.441935   52.345878   

       q_sumprecip   sunHour FIRE_NAME  CAUSE    lat    long  GIS_ACRES  y

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

### Defining Features & Train/Test Split

In [ ]:
feature_cols = [
    "maxtempF", "mintempF", "avgtempF", "totalSnow",
    "humid", "wind", "precip",
    "q_avgtempF", "q_avghumid", "q_sumprecip",
    "sunHour", "lat", "long"
]

X = df[feature_cols].copy()
y_class = df["fire_occurred"]      # for classification
y_reg = df["GIS_ACRES"]            # for regression

# Classification split (stratified)
X_train, X_test, y_train_cls, y_test_cls = train_test_split(
    X, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Regression split (we will refine for GIS_ACRES > 0 later)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X, y_reg, test_size=0.2, random_state=42
)

# Scaling for models that need it
scaler_cls = StandardScaler()
X_train_scaled = scaler_cls.fit_transform(X_train)
X_test_scaled = scaler_cls.transform(X_test)

scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

scaler_full = StandardScaler()
X_scaled_full = scaler_full.fit_transform(X)  # for clustering

print("\nScaled training feature snapshot (classification):")
print(pd.DataFrame(X_train_scaled, columns=feature_cols).head())

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


Scaled training feature snapshot (classification):
   maxtempF  mintempF  avgtempF  totalSnow     humid      wind    precip  \
0 -1.037830 -0.513217 -0.946221  -0.205881  1.049804  0.527325  0.249126   
1  0.086433 -0.783508 -0.165965  -0.205881 -0.141913 -0.907286 -0.511060   
2  1.774550  1.457580  1.755093  -0.205881 -1.613292 -1.396294 -0.534888   
3 -0.262927  0.741439  0.042335  -0.205881  1.509550  1.394010 -0.534888   
4  0.287150 -0.203735  0.263298  -0.205881  0.082810 -0.557794 -0.153642   

   q_avgtempF  q_avghumid  q_sumprecip   sunHour       lat      long  
0   -0.997799    0.035728    -0.457163 -1.942915 -0.988019  0.068027  
1    0.747576    0.168245    -0.668366  0.133353  1.721370 -0.986318  
2    0.094225    0.136836     0.358739  0.138090 -0.389332  0.468476  
3    1.207111   -0.293628    -0.613283  0.101420  0.034557 -1.239767  
4    1.385058   -0.552103    -0.604153  1.415366  1.721370 -0.986318  


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## MODEL 1 – Frequent Pattern Mining (Apriori)

### Why this model was chosen
We used the Apriori algorithm on days where a fire actually occurred to discover common combinations of weather conditions that tend to appear together during fire events. Our dataset contains continuous weather variables (temperature, humidity, wind, precipitation), which we discretized into bins (e.g., temp_high, humid_low). This makes the data suitable for frequent pattern mining and produces interpretable rules like “high temperature + low humidity → fire-prone conditions.”

### Model assumptions

- Each row (day/location) acts like a transaction with a set of weather-condition “items.”

- The relationships between these discretized conditions are stable over time.

- The chosen bins for temperature, humidity, wind, and precipitation capture meaningful thresholds for fire risk.

### Hyperparameter tuning

- We experimented with minimum support values and settled around 0.05, which balances:

  - Too high support → very few rules

  - Too low support → too many noisy patterns

- We filtered rules using lift ≥ 1.0, so we keep only those that improve over random co-occurrence.

### Challenges & solutions

- Challenge: The dataset is mostly continuous and not directly usable as transactions.
Solution: We binned weather features (e.g., avgtempF → temp_low / temp_moderate / temp_high / temp_very_high) and created a one-hot encoded basket using pd.get_dummies.

- Challenge: Too many trivial rules at low support.
Solution: Increased min_support and filtered by confidence and lift to keep only the most meaningful rules.

### Performance / metrics

- We evaluated patterns using Support, Confidence, and Lift from the association rules.

- High-lift rules highlight combinations like high temperature + low humidity that are more common during wildfire events than expected by chance.

In [ ]:
df_fires = df[df["fire_occurred"] == 1].copy()

df_fires["temp_bin"] = pd.cut(
    df_fires["avgtempF"],
    bins=[-np.inf, 60, 75, 90, np.inf],
    labels=["temp_low", "temp_moderate", "temp_high", "temp_very_high"]
)

df_fires["humid_bin"] = pd.cut(
    df_fires["humid"],
    bins=[-np.inf, 30, 60, np.inf],
    labels=["humid_low", "humid_medium", "humid_high"]
)

df_fires["wind_bin"] = pd.cut(
    df_fires["wind"],
    bins=[-np.inf, 5, 15, np.inf],
    labels=["wind_low", "wind_medium", "wind_high"]
)

df_fires["precip_bin"] = pd.cut(
    df_fires["q_sumprecip"],
    bins=[-np.inf, 0.1, 1, np.inf],
    labels=["precip_very_low", "precip_low", "precip_high"]
)

# One-hot encode bins to create "transactions"
basket = pd.get_dummies(df_fires[["temp_bin", "humid_bin", "wind_bin", "precip_bin"]])

print("\nFrequent pattern mining basket snapshot:")
print(basket.head())

# Run Apriori
frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

# Association rules with lift
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

print("\nTop 10 association rules:")
print(
    rules.sort_values("lift", ascending=False)
         .head(10)[["antecedents", "consequents", "support", "confidence", "lift"]]
)


Frequent pattern mining basket snapshot:
      temp_bin_temp_low  temp_bin_temp_moderate  temp_bin_temp_high  \
1238               True                   False               False   
1375               True                   False               False   
1378               True                   False               False   
1398               True                   False               False   
1465              False                    True               False   

      temp_bin_temp_very_high  humid_bin_humid_low  humid_bin_humid_medium  \
1238                    False                False                   False   
1375                    False                False                   False   
1378                    False                False                    True   
1398                    False                False                   False   
1465                    False                False                    True   

      humid_bin_humid_high  wind_bin_wind_low  wind_bin_wind_m

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
# Save frequent itemsets and rules as pickle files
with open("frequent_itemsets.pkl", "wb") as f:
    pickle.dump(frequent_itemsets, f)

with open("association_rules.pkl", "wb") as f:
    pickle.dump(rules, f)

print("\nSaved: frequent_itemsets.pkl and association_rules.pkl")


Saved: frequent_itemsets.pkl and association_rules.pkl


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## MODEL 2 – Classification (SVM)

### Why this model was chosen
Our target fire_occurred is binary (fire vs no fire), and the input features are all numerical after preprocessing and scaling. SVM with an RBF kernel is well-suited for such data because it can model non-linear decision boundaries and works well in medium-dimensional feature spaces like ours (weather + location features).

### Model assumptions

- Features are numerical and scaled (we used StandardScaler).

- Classes are approximately separable in some transformed (kernel) space.

- The decision boundary can be captured by adjusting the penalty parameter C and kernel parameter gamma.

### Hyperparameter tuning
We used GridSearchCV with 3-fold cross-validation over:

- C: [0.1, 1, 10]

- kernel: ["rbf", "linear"]

- gamma: ["scale", 0.1, 0.01]

The grid search picked the combination that maximized F1-score, which is appropriate because our dataset is somewhat imbalanced (much more “no_fire” than “fire” days).

### Challenges & solutions

- Challenge: Class imbalance can make accuracy misleading.
Solution: We used F1-score as the tuning metric and reported precision, recall, F1, and ROC-AUC, giving a better view of how well the model detects fire days.

- Challenge: SVM is sensitive to feature scaling.
Solution: Applied StandardScaler to all input features and saved the scaler as svm_scaler.pkl along with the model (svm_model.pkl) for consistent preprocessing during deployment.

### Performance / metrics (test set)

- Accuracy: ~0.85

- Precision: ~0.80

- Recall: ~0.79

- F1-score: ~0.80

- ROC-AUC: ~0.91

These results show that SVM performs strongly on the main task of predicting whether a fire will occur on a given day based on weather and location.

In [ ]:
svm = SVC(probability=True, random_state=42)

param_grid_svm = {
    "C": [0.1, 1, 10],
    "kernel": ["rbf", "linear"],
    "gamma": ["scale", 0.1, 0.01]
}

grid_svm = GridSearchCV(
    svm,
    param_grid_svm,
    cv=3,
    scoring="f1",    # good for imbalanced data
    n_jobs=-1,
    verbose=1
)

grid_svm.fit(X_train_scaled, y_train_cls)

print("\nBest SVM parameters:", grid_svm.best_params_)
best_svm = grid_svm.best_estimator_

y_pred_cls = best_svm.predict(X_test_scaled)
y_proba_cls = best_svm.predict_proba(X_test_scaled)[:, 1]

acc = accuracy_score(y_test_cls, y_pred_cls)
prec = precision_score(y_test_cls, y_pred_cls)
rec = recall_score(y_test_cls, y_pred_cls)
f1 = f1_score(y_test_cls, y_pred_cls)
roc_auc = roc_auc_score(y_test_cls, y_proba_cls)

print("\n=== SVM Classification Performance ===")
print("Accuracy :", acc)
print("Precision:", prec)
print("Recall   :", rec)
print("F1-score :", f1)
print("ROC-AUC  :", roc_auc)
print("\nClassification report:")
print(classification_report(y_test_cls, y_pred_cls))

print("Confusion matrix:")
print(confusion_matrix(y_test_cls, y_pred_cls))

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Fitting 3 folds for each of 18 candidates, totalling 54 fits


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replac


Best SVM parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


=== SVM Classification Performance ===
Accuracy : 0.8504098360655737
Precision: 0.7969187675070029
Recall   : 0.7946927374301676
F1-score : 0.7958041958041958
ROC-AUC  : 0.9103913552457918

Classification report:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      1236
           1       0.80      0.79      0.80       716

    accuracy                           0.85      1952
   macro avg       0.84      0.84      0.84      1952
weighted avg       0.85      0.85      0.85      1952

Confusion matrix:
[[1091  145]
 [ 147  569]]


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
# Save SVM model
with open("svm_model.pkl", "wb") as f:
    pickle.dump(best_svm, f)

# Save the scaler used for classification
with open("svm_scaler.pkl", "wb") as f:
    pickle.dump(scaler_cls, f)

print("\nSaved: svm_model.pkl and svm_scaler.pkl")


Saved: svm_model.pkl and svm_scaler.pkl


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

# MODEL 3 – Regression (Ridge Regression)

### Why this model was chosen
To model the continuous outcome GIS_ACRES (burned area for fire events), we used Ridge Regression, a form of linear regression with L2 regularization. It serves as a simple, interpretable baseline and is useful when features are correlated (e.g., different temperature or humidity-related variables).

We restricted this model to rows where GIS_ACRES > 0 (actual fires).

### Model assumptions

- Approximate linear relationship between features (weather, location, etc.) and burned area.

- Errors are roughly independent and homoscedastic.

- Feature correlations exist, and L2 regularization helps stabilize coefficients.

### Hyperparameter tuning
We used GridSearchCV over the regularization strength alpha:

- alpha: [0.1, 1.0, 10.0, 100.0]

We selected the alpha value that minimized Mean Squared Error (MSE) in cross-validation.

### Challenges & solutions

- Challenge: The R² score is low (~0.015), meaning the model explains very little of the variance in burned area.
Reason: Burned area is influenced by many factors not in our dataset (fuel type, suppression, terrain, etc.) so weather alone is not enough.
Solution: We still report this as a baseline and highlight that predicting fire occurrence is more reliable than predicting burned area size with the current features. Ridge helps avoid overfitting but cannot fully fix missing information.

### Performance / metrics (test set)

- MSE: ≈ 4.97 × 10⁸

- RMSE: ≈ 22,283 acres

- R²: ≈ 0.015

Interpretation: The model has limited predictive power for fire size; this is an important finding for the project.

In [ ]:
# Use only rows where fire area > 0
reg_mask = df["GIS_ACRES"] > 0
X_reg_all = df.loc[reg_mask, feature_cols]
y_reg_all = df.loc[reg_mask, "GIS_ACRES"]

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg_all, y_reg_all, test_size=0.2, random_state=42
)

scaler_reg2 = StandardScaler()
X_train_reg_scaled = scaler_reg2.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg2.transform(X_test_reg)

ridge = Ridge()

param_grid_ridge = {
    "alpha": [0.1, 1.0, 10.0, 100.0]
}

grid_ridge = GridSearchCV(
    ridge,
    param_grid_ridge,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

grid_ridge.fit(X_train_reg_scaled, y_train_reg)

print("\nBest Ridge parameters:", grid_ridge.best_params_)
best_ridge = grid_ridge.best_estimator_

y_pred_reg = best_ridge.predict(X_test_reg_scaled)

mse = mean_squared_error(y_test_reg, y_pred_reg)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_reg, y_pred_reg)

print("\n=== Ridge Regression Performance ===")
print("MSE :", mse)
print("RMSE:", rmse)
print("R^2 :", r2)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


Best Ridge parameters: {'alpha': 100.0}

=== Ridge Regression Performance ===
MSE : 496537990.9623959
RMSE: 22283.132431558988
R^2 : 0.015184509840126426


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
with open("ridge_model.pkl", "wb") as f:
    pickle.dump(best_ridge, f)

with open("ridge_scaler.pkl", "wb") as f:
    pickle.dump(scaler_reg2, f)

print("\nSaved: ridge_model.pkl and ridge_scaler.pkl")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


Saved: ridge_model.pkl and ridge_scaler.pkl


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## MODEL 4 – Clustering (K-Means)

### Why this model was chosen
K-Means clustering was used to group days/locations into similar weather/condition clusters without using any target labels. This helps us see if there are natural groups such as “fire-prone weather patterns” vs “low-risk weather patterns.”

### Model assumptions

- Clusters are roughly spherical and of similar size in the scaled feature space.

- Uses Euclidean distance, so features must be scaled (we used StandardScaler and saved it as kmeans_scaler.pkl).

### Hyperparameter tuning
We treated the number of clusters k as a hyperparameter and evaluated:

- k = 2, 3, 4, 5, 6

For each k, we computed:

- Silhouette Score (higher is better)

- Davies–Bouldin Index (lower is better)

The best compromise was obtained at k = 2, which had the highest silhouette score among the tested values. We then trained a final K-Means model with k=2 and saved it as kmeans_model.pkl.

### Challenges & solutions

- Challenge: Choosing the right number of clusters.
Solution: Used Silhouette and Davies–Bouldin metrics instead of guessing k.

- Challenge: Different feature scales can distort distances.
Solution: Applied StandardScaler to all features before clustering.

### Performance / metrics

- For each k (2–6), we reported:

  - Silhouette Score (e.g., best ≈ 0.236 for k=2)

  - Davies–Bouldin Index (e.g., DBI ≈ 1.61 for k=2)

The clustering suggests that the dataset can be meaningfully separated into about two main groups of conditions.

In [ ]:
X_clust = X_scaled_full

sil_scores = {}
dbi_scores = {}

for k in range(2, 7):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_clust)
    sil = silhouette_score(X_clust, labels)
    dbi = davies_bouldin_score(X_clust, labels)
    sil_scores[k] = sil
    dbi_scores[k] = dbi
    print(f"KMeans k={k}: Silhouette={sil:.3f}, Davies-Bouldin={dbi:.3f}")

best_k = max(sil_scores, key=sil_scores.get)
print(f"\nBest k based on Silhouette score: {best_k}")

kmeans_best = KMeans(n_clusters=best_k, random_state=42, n_init=10)
df["cluster"] = kmeans_best.fit_predict(X_clust)

print("\nCluster counts:")
print(df["cluster"].value_counts())


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

KMeans k=2: Silhouette=0.236, Davies-Bouldin=1.612


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

KMeans k=3: Silhouette=0.161, Davies-Bouldin=1.826


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

KMeans k=5: Silhouette=0.171, Davies-Bouldin=1.506


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

KMeans k=6: Silhouette=0.152, Davies-Bouldin=1.612

Best k based on Silhouette score: 2

Cluster counts:
cluster
0    5613
1    4143
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
with open("kmeans_model.pkl", "wb") as f:
    pickle.dump(kmeans_best, f)

with open("kmeans_scaler.pkl", "wb") as f:
    pickle.dump(scaler_full, f)

print("\nSaved: kmeans_model.pkl and kmeans_scaler.pkl")


Saved: kmeans_model.pkl and kmeans_scaler.pkl


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Summary Tables

In [ ]:
cls_metrics = pd.DataFrame({
    "Model": ["SVM"],
    "Accuracy": [acc],
    "Precision": [prec],
    "Recall": [rec],
    "F1-score": [f1],
    "ROC-AUC": [roc_auc]
})
print("\nClassification metrics summary:")
print(cls_metrics)

reg_metrics = pd.DataFrame({
    "Model": ["Ridge Regression"],
    "MSE": [mse],
    "RMSE": [rmse],
    "R2": [r2]
})
print("\nRegression metrics summary:")
print(reg_metrics)

clust_metrics = pd.DataFrame({
    "k": list(sil_scores.keys()),
    "Silhouette": list(sil_scores.values()),
    "Davies-Bouldin": [dbi_scores[k] for k in sil_scores.keys()]
})
print("\nClustering metrics summary:")
print(clust_metrics)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


Classification metrics summary:
  Model  Accuracy  Precision    Recall  F1-score   ROC-AUC
0   SVM   0.85041   0.796919  0.794693  0.795804  0.910391

Regression metrics summary:
              Model           MSE          RMSE        R2
0  Ridge Regression  4.965380e+08  22283.132432  0.015185

Clustering metrics summary:
   k  Silhouette  Davies-Bouldin
0  2    0.235764        1.612302
1  3    0.161228        1.825516
2  4    0.169477        1.689122
3  5    0.170803        1.506173
4  6    0.152315        1.612290


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

For the main prediction task (fire vs no fire), the SVM classifier clearly works best:

- High ROC-AUC (~ 0.91) and balanced precision/recall (~ 0.80) indicate strong discriminative performance.

For predicting burned area, Ridge Regression shows that:

- Weather and location alone do not explain much variance in fire size (R² ≈ 0.015).

- This suggests that additional features (fuel type, suppression resources, terrain, etc.) are needed for better area prediction.

K-Means clustering doesn’t directly “predict” outcomes but reveals:

- Moderate structure in the data (best clustering around k=2), separating conditions into a small number of typical patterns.

Frequent Pattern Mining (Apriori) does not output accuracy or error metrics but:

- Provides interpretable rules that help explain why certain conditions are risky (e.g., combinations of high temperature, low humidity, and low precipitation).

### Overall conclusion:

- The SVM classification model is the most effective approach for predicting wildfire occurrence given the current dataset.

- The Ridge Regression model is useful as a baseline but highlights that predicting burned area is challenging with only weather features.

- K-Means and Apriori complement these models by giving unsupervised structure and interpretable condition rules that can be used in risk analysis and communication.